https://medium.com/towards-data-science/twitter-sentiment-analysis-on-car-brands-c13d449eb9fc
https://www.ravikiranj.net/posts/2012/code/how-build-twitter-sentiment-analyzer/
http://textblob.readthedocs.io/en/latest/advanced_usage.html#sentiment-analyzers
https://marcobonzanini.com/2015/05/17/mining-twitter-data-with-python-part-6-sentiment-analysis-basics/
http://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/
https://dev.to/rodolfoferro/sentiment-analysis-on-trumpss-tweets-using-python-


In [1]:
# -*- coding: utf-8 -*-
"""
    @author: jose 
    FI UNAM
    Created on Thu Oct  26 21:22:06 2017
"""

from pymongo import MongoClient
from nltk.tokenize import TweetTokenizer
import re

In [2]:
try:
    client = MongoClient()
    print("Connected to MongoDB\n")
except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB",e)

Connected to MongoDB



# Obtener tweets de DB

In [3]:
db = client.nov5_db
tweets = db.nov5_collection

In [59]:
tknzr = TweetTokenizer(preserve_case=False,       # Convertir a minúsculas
                       reduce_len=True,           # Reducir caracteres repetidos
                       strip_handles=False)        # Mostrar @usuarios

lista_de_paro = ['"',"“","/","(",")","[","]","?","¿","!","¡",
                 "rt",":","…",",","\n","#","@","\t","",".","$",
                 "...","-","🤢"]

In [60]:
def clean_tweet(tweet):
    """
    Eliminar tokens que:
    - Estén dentro de lista_de_paro.
    - Sean ligas.
    - Si es una mención i.e @potus, se cambia por token genérico @usuario.
    """
    return [re.sub(r'#([^\s]+)', r'\1', _) if not _.startswith('@')
            else '@usuario'
            for _ in tmp if _ not in lista_de_paro
            and not _.startswith(('http', 'htt'))]           

# Etiquetado

In [82]:
corpus = set()
dataset = open("tweets_etiquetados","w")

for i in tweets.find():
    tag = []
    tmp = tknzr.tokenize(i["text"])
    tmp = clean_tweet(tmp)
    print(' '.join(tmp))
    sentimiento = input("Positivo - 1\nNegativo - 0 \n")
    tag.append(' '.join(tmp))
    tag.append(" "+sentimiento)
    dataset.write(''.join(tag)+'\n')
    print(tag)
    print()

dataset.close()

extraño jugar videojuegos
Positivo - 1
Negativo - 0 
0
['extraño jugar videojuegos', ' 0']

@usuario if your temperature is wrong they get cooked too fast it's annoying
Positivo - 1
Negativo - 0 
0
["@usuario if your temperature is wrong they get cooked too fast it's annoying", ' 0']

de amor nadie se muere pero como duelen las heridas
Positivo - 1
Negativo - 0 
0
['de amor nadie se muere pero como duelen las heridas', ' 0']

fyi there is no daylight saving time in mexico
Positivo - 1
Negativo - 0 
1
['fyi there is no daylight saving time in mexico', ' 1']

por dónde vivirá chema yazpik
Positivo - 1
Negativo - 0 
1
['por dónde vivirá chema yazpik', ' 1']

@usuario awww cosita me preocupa la poca creatividad de las nuevas generaciones
Positivo - 1
Negativo - 0 
0
['@usuario awww cosita me preocupa la poca creatividad de las nuevas generaciones', ' 0']

moronguita ringo shelvy heredera en ajusco distrito federal
Positivo - 1
Negativo - 0 
1
['moronguita ringo shelvy heredera en ajusco di

# Preprocesamiento de tweets y entrenamiento

In [80]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes as NB
from sklearn.metrics import roc_auc_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


SEED = 42

In [81]:
dataset = open("tweets_etiquetados","r")

X,Y = [],[]

for i,tweet in enumerate(dataset):
    tmp = tweet.split()
    tmp = [_ for _ in tmp if _ not in ['@usuario']]
    y = tmp.pop()
    x = tmp
    X.append(' '.join(x))
    Y.append(y)
    print(i,x,y)

0 ['extraño', 'jugar', 'videojuegos'] 0
1 ['if', 'your', 'temperature', 'is', 'wrong', 'they', 'get', 'cooked', 'too', 'fast', "it's", 'annoying'] 0
2 ['de', 'amor', 'nadie', 'se', 'muere', 'pero', 'como', 'duelen', 'las', 'heridas'] 0
3 ['fyi', 'there', 'is', 'no', 'daylight', 'saving', 'time', 'in', 'mexico'] 1
4 ['por', 'dónde', 'vivirá', 'chema', 'yazpik'] 1
5 ['awww', 'cosita', 'me', 'preocupa', 'la', 'poca', 'creatividad', 'de', 'las', 'nuevas', 'generaciones'] 0
6 ['moronguita', 'ringo', 'shelvy', 'heredera', 'en', 'ajusco', 'distrito', 'federal'] 1
7 ['pan', 'orgánico', 'de', 'masa', 'madre', 'de', 'centeno', 'o', 'integral', 'panorganico', 'masamadre', 'organicbread'] 1
8 ['topsemanal', 'kate', 'del', 'castillo', 'estrena', 'novio'] 1
9 ['last', 'night', '😊', '😆', '😍', '😉', 'en', 'mexico', 'city', 'mexico'] 1
10 ['llueve', 'otra', 'vez', 'silvio', 'rodríguez', 'vía', 'mi', 'canción', 'favorita', 'del', 'gran', 'silviorodríguez'] 1
11 ['buen', 'día', 'excelente', 'exposición', 

In [82]:
data = pd.DataFrame()

In [83]:
data["tweet"] = X

In [84]:
data["tag"] = Y

In [85]:
data.head(50)

,tweet,tag
0,extraño jugar videojuegos,0
1,if your temperature is wrong they get cooked t...,0
2,de amor nadie se muere pero como duelen las he...,0
3,fyi there is no daylight saving time in mexico,1
4,por dónde vivirá chema yazpik,1
5,awww cosita me preocupa la poca creatividad de...,0
6,moronguita ringo shelvy heredera en ajusco dis...,1
7,pan orgánico de masa madre de centeno o integr...,1
8,topsemanal kate del castillo estrena novio,1
9,last night 😊 😆 😍 😉 en mexico city mexico,1


In [86]:
stopset = set(stopwords.words('spanish'))
vectorizer = TfidfVectorizer(use_idf=True,
                             lowercase=True,
                             strip_accents='ascii',
                             stop_words=stopset,
                             smooth_idf=True)

In [87]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'hayan', 'suyo', 'estará', 'tendríamos', 'los', 'tanto', 'nuestros', 'sentidas', 'tenías', 'fuésemos', 'en', 'serás', 'tendrás', 'estaríais', 'tuvieran', 'tuvieron', 'míos', 'y', 'tuvo', 'ellos', 'hubieran', 'esta', 'fueses', 'estados', 'está', 'estuvimos', 'hubierais', 'fuiste', 'estaba...seréis', 'le', 'estada', 'estaría', 'fueseis', 'como', 'estén', 'seremos', 'tuyos', 'vuestro', 'su'},
        strip_accents='ascii', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [88]:
y = data.tag

In [89]:
X = vectorizer.fit_transform(data.tweet)

In [90]:
from collections import Counter
print(Counter(y))

Counter({'1': 665, '0': 482})


In [91]:
print(X.shape,y.shape)

(1147, 3574) (1147,)


In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42,)

In [93]:
clf = NB.MultinomialNB()
clf.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [94]:
arbol = RandomForestClassifier(random_state = SEED,
                               criterion = 'entropy',
                               max_depth = 50)
arbol.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [95]:
soporte = SVC(C=60,
              random_state = SEED,
              kernel='poly',
              degree=4)
soporte.fit(X_train,y_train)

SVC(C=60, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=4, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [96]:
arbol.score(X_test,y_test)

0.64459930313588854

In [97]:
clf.score(X_test,y_test)

0.71080139372822304

In [98]:
soporte.score(X_test,y_test)

0.59581881533101044

In [99]:
new_tweet = np.array(["no te voy a estar rogando"])

new_tweet_vector = vectorizer.transform(new_tweet)
print(new_tweet_vector)
print(clf.predict(new_tweet_vector))
print(arbol.predict(new_tweet_vector))

  (0, 3526)	1.0
['0']
['1']


In [100]:
arbol.decision_path

<bound method BaseForest.decision_path of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)>